In [12]:
#r "nuget: Tomlyn, 0.17.0"

using System;
using System.IO;
using System.Text.RegularExpressions;
using Tomlyn;
using Tomlyn.Model;

List<(string UUID, string RootUUID, string Name)> loadListFromTOMLFile(string filePath, string listName) // TOML proper
{
    if (!File.Exists(filePath))
        return null;

    var parsedLines = new List<(string, string, string)>();

    var model = Toml.ToModel(File.ReadAllText(filePath));
    var inputArray = (TomlArray)((TomlTable)model["CoreObjectCache"]!)[listName];

    foreach (var entry in inputArray)
    {
        // Entries are an array within an array
        var a = (TomlArray)entry;
        var (x, y, z) = ((string)a[0], (string)a[1], (string)a[2]);

        // Manually trim appended numbers
        var m = Regex.Matches(z.Trim(), @"^(.*?)( \d+)?$");

        parsedLines.Add((x, y, m[0].Groups[1].Value));
    }

    return parsedLines;
}

List<(string UUID, string RootUUID, string Name)> loadRawListFromFile(string filePath) // [ "X", "Y", "Z" ],
{
    if (!File.Exists(filePath))
        return null;

    var parsedLines = new List<(string, string, string)>();
    var pattern = @"\[ ""(.*?)"", ""(.*?)"", ""(.*?)( \d+)?"" \],";

    foreach (var line in File.ReadAllLines(filePath))
    {
        var matches = Regex.Matches(line.Trim(), pattern);

        if (!matches.Any())
            continue;

        var match = matches[0];
        parsedLines.Add((match.Groups[1].Value, match.Groups[2].Value, match.Groups[3].Value));
    }

    return parsedLines;
}

// Merge entries when the primary list contains "X" for a name
var primaryList = loadListFromTOMLFile(Path.Combine(Environment.CurrentDirectory, "mod_config.ini"), "CachedSpawnSetups");
mergeEntityListFromFile(Path.Combine(Environment.CurrentDirectory, "merge_list_source.txt"), false);
mergeEntityListFromFile(Path.Combine(Environment.CurrentDirectory, "merge_list_source_names.txt"), true);

void mergeEntityListFromFile(string filePath, bool namesOnly)
{
    var mergeSourceList = loadRawListFromFile(filePath);

    if (mergeSourceList == null)
        return;

    Console.WriteLine($"Merging file with {mergeSourceList.Count} entries...");

    // If one UUID has multiple different names, reset the name back to "X"
    var filteredUUIDNames = new Dictionary<string, string>();
    var badUUIDNames = new HashSet<string>();

    foreach (var sourceEntry in mergeSourceList)
    {
        if (filteredUUIDNames.TryGetValue(sourceEntry.UUID, out var storedName))
        {
            if (storedName != sourceEntry.Name)
                badUUIDNames.Add(sourceEntry.UUID);
        }
        else
            filteredUUIDNames.Add(sourceEntry.UUID, sourceEntry.Name);
    }

    for (int i = 0; i < mergeSourceList.Count; i++)
    {
        if (badUUIDNames.Contains(mergeSourceList[i].UUID))
            mergeSourceList[i] = (mergeSourceList[i].UUID, mergeSourceList[i].RootUUID, "X");
    }

    // The incoming merge list may contain duplicate UUIDs belonging to different RootUUIDs. We'll use the RootUUID containing
    // the fewest number of UUIDs.
    var byHash = new Dictionary<string, (string UUID, string RootUUID, string Name)>();
    var bySmallestRootUUIDCount = mergeSourceList
        .GroupBy(x => x.RootUUID)
        .ToList()
        .OrderBy(x => x.Count());

    foreach (var group in bySmallestRootUUIDCount)
    {
        foreach (var sourceEntry in group)
            byHash.TryAdd(sourceEntry.UUID, sourceEntry);
    }

    // Keep the primary list intact by copying over names first
    for (int i = 0; i < primaryList.Count; i++)
    {
        if (primaryList[i].Name == "X" && byHash.TryGetValue(primaryList[i].UUID, out var source))
        {
            if (namesOnly)
                primaryList[i] = (primaryList[i].UUID, primaryList[i].RootUUID, source.Name);
            else
                primaryList[i] = source;
        }

        byHash.Remove(primaryList[i].UUID);
    }

    // Then append unmatched remainder
    if (!namesOnly)
    {
        foreach (var (key, value) in byHash)
            primaryList.Add(value);
    }
}

// Count duplicate names
var outputNameCounters = new Dictionary<string, int>();
var nameCounts = primaryList
    .Select(x => x.Name)
    .Where(x => x.Length > 1)
    .GroupBy(x => x)
    .ToDictionary(x => x.Key, y => y.Count());

string getFormattedName(string Name)
{
    if (Name.Length <= 1 || nameCounts[Name] <= 1)
        return Name;

    if (!outputNameCounters.TryAdd(Name, 1))
        outputNameCounters[Name]++;

    return $"{Name} {outputNameCounters[Name]}";
}

// Them dump back to disk
var outputFilePath = Path.Combine(Environment.CurrentDirectory, "reformatted_entity_list.txt");

File.WriteAllLines(outputFilePath, primaryList
    .OrderBy(x => x.UUID)
    .Select(x => $"    [ \"{x.UUID}\", \"{x.RootUUID}\", \"{getFormattedName(x.Name)}\" ],")
    .Append("]")
    .Prepend("    # [ UUID, Root UUID, Name ]")
    .Prepend("CachedSpawnSetups = ["));

Installed Packages Tomlyn, 0.17.0